In [ ]:
from typing import Dict, Any, Union
import json
from plaid.api import plaid_api
from plaid.model.country_code import CountryCode
from plaid.model.link_token_create_request_user import LinkTokenCreateRequestUser
from test_configuration import plaid_client, firestore_client
from plaid import ApiException

from plaid.model.sandbox_item_reset_login_request import SandboxItemResetLoginRequest
from plaid.model.sandbox_item_reset_login_response import SandboxItemResetLoginResponse

from google.cloud.firestore import CollectionReference

In [ ]:
def get_access_token_to_update(uid: str, institution_id: str):
    try:
        user_ref = firestore_client.collection('users').document(uid)
        secrets_ref: CollectionReference = user_ref.collection('secrets')
        plaid_secrets_ref = secrets_ref.document('plaid')
        plaid_secrets_dict = plaid_secrets_ref.get().to_dict()
        access_tokens: Union[Dict, None] = plaid_secrets_dict.get('access_tokens')
        access_token = access_tokens.get(institution_id)

        return {'status': 200, 'access_token': access_token}
    except Exception as e:
        return {'status': 404, 'error_message': str(e)}

In [ ]:
''' Force a Sandbox Item into an error state

    `/sandbox/item/reset_login/` forces an Item into an `ITEM_LOGIN_REQUIRED` 
    state in order to simulate an Item whose login is no longer valid. This makes 
    it easy to test Link's update mode flow in the Sandbox environment. After calling 
    `/sandbox/item/reset_login`, You can then use Plaid Link update mode to restore 
    the Item to a good state. An `ITEM_LOGIN_REQUIRED` webhook will also be fired 
    after a call to this endpoint, if one is associated with the Item.

    In the Sandbox, Items will transition to an `ITEM_LOGIN_REQUIRED` error state 
    automatically after 30 days, even if this endpoint is not called.
'''


def sandbox_item_reset_login(access_token: str):
    try:
        request = SandboxItemResetLoginRequest(access_token)
        response: SandboxItemResetLoginResponse = plaid_client.sandbox_item_reset_login(
            request
        )
        return response.to_dict()
    except ApiException as e:
        exceptions: dict = json.loads(e.body)
        return exceptions

In [ ]:
sandbox_item_reset_login('access_token')

In [ ]:
def link_token_create_update_mode(data: Dict) -> Dict[str, Any]:
    # TODO: ^change the data type to flask.Request for release

    # # TODO: for development
    # # Get `uid` and `institution_id` from request
    # data_dict: dict = json.loads(request.data)
    # uid = data_dict.get('uid')
    # institution_id = data_dict.get('institution_id')

    # TODO: for sandbox
    uid = data.get('uid')
    institution_id = data.get('institution_id')

    if (uid and institution_id) is None:
        return {'status': 404, 'error_message': 'Either access_token, uid, institution_id was None. Please send the data correctly'}
        
    access_token_response = get_access_token_to_update(uid, institution_id)
    access_token = access_token_response.get('access_token')
    
    if access_token is None:
        return {'status': 404, 'error_message': access_token_response.get('error_message')}

    try:
        # Create Link Token Request
        request = plaid_api.LinkTokenCreateRequest(
            client_name='CCCC',
            language='en',
            access_token = access_token,
            country_codes=[CountryCode('US')],
            user=LinkTokenCreateRequestUser(client_user_id=uid)
        )
        response: plaid_api.LinkTokenCreateResponse = plaid_client.link_token_create(request)

        return response.to_dict()
    except ApiException as e:
        return json.loads(e.body)


In [ ]:
data = {
    'uid': 'uid',
    'institution_id': 'ins_10'
}

link_token_create_update_mode(data)